In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pysumma.plotting as psp
import seaborn as sns
import matplotlib.pyplot as plt

import sys
import xarray as xr
import pandas as pd
import pysumma as ps

# function to convert summa
def convert_time_to_summa_string(t):
    return (
        f'{t.dt.year.values[()]:04}'
        f'-{t.dt.month.values[()]:02}'
        f'-{t.dt.day.values[()]:02}'
        f' {t.dt.hour.values[()]:02}'
        f':{t.dt.minute.values[()]:02}'
    )
# set up attributes for forcing data
attrs = {
   'airpres':  {'units': 'Pa', 'long_name': 'Air pressure'},
   'airtemp':  {'units': 'K', 'long_name': 'Air temperature'},
   'spechum':  {'units': 'g g-1', 'long_name': 'Specific humidity'},
   'windspd':  {'units': 'Wind speed', 'long_name': 'm s-1'},
   'SWRadAtm': {'units': 'W m-2', 'long_name': 'Downward shortwave radiation'},
   'LWRadAtm': {'units': 'W m-2', 'long_name': 'Downward longwave radiation'},
   'pptrate':  {'units': 'kg m-2 s-1', 'long_name': 'Precipitation rate'}
}
name_lookup = {
    'airpres':  'pressure_Pa',
    'airtemp':  'temp_K',
    'spechum':  'spechum',
    'windspd':  'Wind_ms',
    'SWRadAtm': 'SW_wm2',
    'LWRadAtm': 'LW_wm2',
    'pptrate':  'precip_mm',
}


# Check to see if met data is in PST or UTC
# FORCING AND WRITE TO NET CDF
df = pd.read_csv('/Users/ianwhidden/pysumma/pysumma/COPY_workspace_HJA_pysumma/hjandrews_summa_setup/met_uplo_020523.csv')
# create date time index, which somehow tells summa that this column in the df is the time object/index to use?
df.index = pd.DatetimeIndex(df['datetime'], name='time')
# Ask Andrew about code in this section. How does the df turn into a netCDF here?
forcing_filename = 'uplo_station_forcing.nc'
# Adding 1 hour to account for SUMMA being period-ending
time_idx = df.index + pd.Timedelta('1H')
shape = (len(time_idx), 1, )
dims = ('time', 'hru', )
coords = {'time': time_idx}

met_data = xr.Dataset(coords=coords)
met_data.time.encoding['calendar'] = 'standard'
met_data.time.encoding['units'] = 'hours since 2013-11-01'
for varname, varattrs in attrs.items():
    df_name = name_lookup[varname]
    met_data[varname] = xr.DataArray(
        data=df[df_name].values.reshape(-1, 1),
        coords=coords, dims=dims, name=varname, attrs=varattrs
    )

met_data['pptrate'] /= 3600.0  # Convert from mm/hr to mm/s AKA kg m-2 s-1
# 3600 is timestep xr.variable so it doesnt end up as cords etc
met_data['data_step'] = xr.Variable([], 3600.0)
met_data.to_netcdf(f'./forcings/{forcing_filename}')

with open('./forcings/forcing_file_list.txt', 'w') as f:
    f.write(f"'{forcing_filename}'\n")

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
# Local attributes

lat = 44.2072180256268
lon = -122.119450090239
elev = 1300
# open local_attributes.nc
local_attrs = xr.open_dataset('/Users/ianwhidden/pysumma/pysumma/COPY_workspace_HJA_pysumma/hjandrews_summa_setup/params/local_attributes.nc').load()

# Update local_attributes with desired values
local_attrs['longitude'].values[:] = lon
local_attrs['latitude'].values[:] = lat
local_attrs['elevation'].values[:] = elev
local_attrs['tan_slope'] = 0.0
local_attrs['mHeight'] = 3.0
# 1 is evergreen needleleaf forest in the MODIFIED_IGBP_MODIS_NOAH option
# for the `vegeParTbl` decision. This can be found in the `VEGPARM.TBL` file
# 7 is Open Shrublands
# 16 is Barren or Sparsely Vegetated'
local_attrs['vegTypeIndex'] = 16
#.values[:]
local_attrs.to_netcdf('/Users/ianwhidden/pysumma/pysumma/COPY_workspace_HJA_pysumma/hjandrews_summa_setup/local_attributes.nc')

#local_attrs

In [3]:
!./install_local_setup.sh

summa_exe = 'summa.exe'
file_manager = '/Users/ianwhidden/pysumma/pysumma/COPY_workspace_HJA_pysumma/hjandrews_summa_setup/file_manager.txt'
# INITIATE (instantiate?) simiulation object
s = ps.Simulation(summa_exe, file_manager)

# modify output, decisions, etc. here, below the instantiation of the simulation object

# Update file manager with start and end time
s.manager['simStartTime'] = '2013-11-01 00:00'
s.manager['simEndTime'] = '2014-06-01 00:00'

s.manager['settingsPath'] = '/Users/ianwhidden/pysumma/pysumma/COPY_workspace_HJA_pysumma/hjandrews_summa_setup/settings/'
s.manager['forcingPath'] = '/Users/ianwhidden/pysumma/pysumma/COPY_workspace_HJA_pysumma/hjandrews_summa_setup/forcings/'
s.manager['outputPath'] = '/Users/ianwhidden/pysumma/pysumma/COPY_workspace_HJA_pysumma/hjandrews_summa_setup/output/'
s.manager['trialParamFile'] = '/Users/ianwhidden/pysumma/pysumma/COPY_workspace_HJA_pysumma/hjandrews_summa_setup/params/parameter_trial.nc/'
# When I edit some of the below, I receive error.
#s.manager['globalHruParamFile'] = '/Users/ianwhidden/pysumma/pysumma/COPY_workspace_HJA_pysumma/hjandrews_summa_setup/params/local_param_info.txt/'
#s.manager['globalGruParamFile'] = '/Users/ianwhidden/pysumma/pysumma/COPY_workspace_HJA_pysumma/hjandrews_summa_setup/params/basin_param_info.txt/'
#s.manager['attributeFile'] = '/Users/ianwhidden/pysumma/pysumma/COPY_workspace_HJA_pysumma/hjandrews_summa_setup/params/local_attributes.nc/'
#s.manager['trialParamFile'] = '/Users/ianwhidden/pysumma/pysumma/COPY_workspace_HJA_pysumma/hjandrews_summa_setup/params/parameter_trial.nc/'

#s.manager['forcingListFile'] = '/Users/ianwhidden/pysumma/pysumma/COPY_workspace_HJA_pysumma/hjandrews_summa_setup/forcings/forcing_file_list.txt/'
#s.manager['initConditionFile'] = '/Users/ianwhidden/pysumma/pysumma/COPY_workspace_HJA_pysumma/hjandrews_summa_setup/params/initial_conditions.nc/'

In [4]:
# Open initial conditions file indendently of summa
#ic = xr.open_dataset('../summa_setup_template/params/initial_conditions.nc').load()
#ic
# ANDREW recomonds to not change initial conditions bc it is a pain. The below code doesnt work
#s.initial_conditions['scalarCanopyTemp']=279.15
#s.initial_conditions
#s.manager

In [5]:
"""
!./install_local_setup.sh

summa_exe = 'summa.exe'
file_manager = './file_manager.txt'
s = ps.Simulation(summa_exe, file_manager)
"""

# modify output, decisions, etc. here, below the instantiation of the simulation object

# isel grabs first and last time stamp.
t0 = met_data['time'].isel(time=0)
t1 = met_data['time'].isel(time=-1)
# 
s.manager['simStartTime'] = convert_time_to_summa_string(t0)
s.manager['simEndTime'] = convert_time_to_summa_string(t1)

# OUTPUT
# Add a variable written to the output control file
s.output_control['scalarSnowDepth'] = [1, 0, 1, 0, 0, 0, 0, 0]
s.output_control['scalarSnowAlbedo'] = [1, 0, 1, 0, 0, 0, 0, 0]

# DECISIONS - Parameter values are going to be much more important than the decisions!
# Andreadis et al. 2009 Includes a rapid interception increase between -3 and 0 C
# from observations of increased cohesion in warm regions.
s.decisions['snowIncept']= 'stickySnow'
print(s.decisions['snowIncept'])

#s.decisions['canopySrad']= 
#s.decisions['snowLayers']= 
#s.decisions['thCondSnow']= 

# set the canopy interception option to sparse canopy
s.decisions.set_option('cIntercept', 'sparseCanopy')


# NEW SNOW DENS - decision doesn't work for now...
#s.decisions.set_option('snowDenNew', 'anderson')
#print(s.decisions['snowDenNew'])
#s.decisions

snowIncept    stickySnow           ! choice of parameterization for snow interception


## Parameters

In [6]:
# (5) Model Parameters -  LOCAL PARAMETERS (s.global_hru_params) - set parameters and view those changes AKA local_params.txt

# SNOW
# Max albedo of 0.85 comes from Andreadis et al., 2009
#s.global_hru_params['albedoMax'] = 0.99
s.global_hru_params['albedoSootLoad'] = 0.00001
print(s.global_hru_params['albedoSootLoad'])


print(s.global_hru_params['albedoMax'])
print(s.global_hru_params['albedoDecayRate'])
print(s.global_hru_params['albedoRefresh'])


# CANOPY
# roughness length of canopy (m)
# Dont worry too much about this to start
#s.global_hru_params['z0Canopy'] = 

# zero plane displacement / canopy height (-)
# related to turbulent fluxes and wind speed scaling. Dont worry about this.
# s.global_hru_params['zpdFraction'] = 

# height of top of the vegetation canopy above ground 
s.global_hru_params['heightCanopyTop'] = 0
# height of bottom of the vegetation canopy above ground surface (m)
s.global_hru_params['heightCanopyBottom'] = 0

print(s.global_hru_params['z0Canopy'])
print(s.global_hru_params['zpdFraction'])
print(s.global_hru_params['heightCanopyTop'])
print(s.global_hru_params['heightCanopyBottom'])

# RADIATION
#s.global_hru_params['Frad_direct'] = 0.2
print(s.global_hru_params['Frad_direct'])
print(s.global_hru_params['Frad_vis'])

# Precipitation partitioning
# critical temperature where precipitation is rain (K)
# 39 F = 277.039 , 36 F = 275.372 k , 32 F = 273.15 K
#s.global_hru_params['tempCritRain']= 277.039
print(s.global_hru_params['tempCritRain'])

# frozen precipitation multiplier (-)
print(s.global_hru_params['frozenPrecipMultip'])

print(s.global_hru_params['refInterceptCapSnow'])


albedoSootLoad            |       1.0d-5 |       1.0d-5 |       1.0d-5
albedoMax                 |       0.8400 |       0.7000 |       0.9500
albedoDecayRate           |       1.0d+6 |       1.0d+5 |       5.0d+6
albedoRefresh             |       1.0000 |       1.0000 |      10.0000
z0Canopy                  |       0.1000 |       0.0010 |      10.0000
zpdFraction               |       0.6500 |       0.5000 |       0.8500
heightCanopyTop           |       0.0000 |       0.0000 |       0.0000
heightCanopyBottom        |       0.0000 |       0.0000 |       0.0000
Frad_direct               |       0.9000 |       0.0000 |       1.0000
Frad_vis                  |       0.9000 |       0.0000 |       1.0000
tempCritRain              |     273.1600 |     272.1600 |     274.1600
frozenPrecipMultip        |       1.0000 |       0.5000 |       1.5000
refInterceptCapSnow       |       6.6000 |       1.0000 |      10.0000


In [7]:
s.run('local')
s.status
#print(s.stderr)

# The output of summa is an xarray dataset

'Success'

In [ ]:
#print(s.stderr)
s.reset()
%reset

In [ ]:
# Write something from output to a csv

dssd = s.output['scalarSnowDepth']
df = dssd.to_dataframe()
df = df.to_csv('/Users/ianwhidden/Desktop/thesis_data/summa_output/s2_sd.csv')

dsswe = s.output['scalarSWE']
df = dsswe.to_dataframe()
df = df.to_csv('/Users/ianwhidden/Desktop/thesis_data/summa_output/s2_swe.csv')



<br>

## Plotting

Now that we've got some output we can plot some results. Because the output is an xarray `DataSet` we can use the convenient plotting capabilities provided by xarray.

In [ ]:
# PLOTS , current is snow albedo

# mass of total water on the vegetation canopy (kg m-2)
#s.output['scalarCanopyWat'].plot(label='SUMMA')

# temperature of each layer (K)
#s.output['mLayerTemp'].plot(label='SUMMA')

#albedo of the ground surface (-)
#s.output['scalarGroundAlbedo'].plot(label='SUMMA')


# Albedo commonly refers to the "whiteness" of a surface, with 0 meaning black and 1 meaning white.
#A value of 0 means the surface is a "perfect absorber" that absorbs all incoming energy.

#snow albedo for the entire spectral band (-)
#s.output['scalarSnowAlbedo'].plot(label='SUMMA')

albedo = s.output['scalarSnowAlbedo']
albedo.plot.line(x='time', color="purple")
# Change the max and min values of plot to show albedo variations
plt.ylim([0, 1])
# Set axis sizes
matplotlib.rc('xtick', labelsize=10) 
matplotlib.rc('ytick', labelsize=10)

#matplotlib.title('Snow Albedo')

#saturation vapor pressure at the temperature of vegetation canopy (Pa)
#s.output['scalarSatVP_CanopyTemp'].plot(label='SUMMA')

#saturation vapor pressure at the temperature of the ground (Pa)
#s.output['scalarSatVP_GroundTemp'].plot(label='SUMMA')

#net radiation (W m-2)
#Graph shows constant net radiation of zero
#s.output['scalarNetRadiation'].plot(label='SUMMA')


#latent heat from the ground (below canopy or non-vegetated) (W m-2)
#s.output['scalarLatHeatGround'].plot(label='SUMMA')


#s.output['scalarLAI'].plot(label='SUMMA')



In [ ]:
fig, axs = plt.subplots(ncols=2)

In [ ]:
depth = s.output['scalarSnowDepth']
depth.plot.line(x='time',label='Snow depth')
#plt.suptitle('Snow depth')
#depth.title('Depth')
#plt.plot(s.output['scalarSnowDepth'], x='time')
#from matplotlib import pyplot as plt    

#fig = plt.figure()
#plt.plot(s.output['scalarSnowDepth'])
#plt.suptitle('Snowpack depth', fontsize=20)

#plt.xlabel('Time', fontsize=18)
#plt.ylabel('Snowpack depth m', fontsize=16)
#fig.savefig('test.jpg')


In [ ]:
# PLOT TEMP IN FARENHEIT - prognostic (predicted by model) - scalarSurfaceTemp is the temp of the ground surface
temp = s.output['scalarSurfaceTemp']

# Convert
temp_f = 1.8*(temp-273) + 32
print(temp_f)
#s.output['scalarSurfaceTemp'].plot(label='SUMMA');
temp_f.plot(label='temp_F')

In [ ]:
s.output['scalarSurfaceTemp'].plot(label='SUMMA');

In [ ]:
# plot for SWE

s.output['scalarSWE'].plot(label='SUMMA');
#s.output['scalarSnowDepth'].plot(label='SUMMA');
plt.legend();

In [ ]:
# plot for something else
"""
s.output['scalarSurfaceTemp'].plot(label='SUMMA');
#s.output['scalarSnowDepth'].plot(label='SUMMA');
plt.legend();
"""
#s.output_control
#s.output['scalarSnowAlbedo'].plot(color='red', linewidth=2);
s.output

In [ ]:
# s.output[whichever cell/data]

# Total snow depth
depth = s.output.isel(hru=0)['iLayerHeight']
temp = s.output.isel(hru=0)['mLayerTemp']
frac_wat = s.output.isel(hru=0)['mLayerVolFracWat']

psp.layers(temp, depth, colormap='viridis', plot_soil=False, plot_snow=True);
s.output['scalarSnowDepth'].plot(color='red', linewidth=2);



In [ ]:
# Volumetric water content

psp.layers(frac_wat, depth, colormap='Blues', plot_soil=True, plot_snow=True);
#s.output['scalarSnowDepth'].plot(color='red', linewidth=2);

# Change what is written in s.output_control (txt)

In [ ]:
# Add a variable written to output

"""s.output_control['scalarSnowDepth'] = {
    'period': 1, 'instant': 1, 'sum': 0, 
    'mean': 0, 'variance': 0, 'min': 0, 'max': 0}
    """
#s.output_control['scalarSnowDepth']


## s.decisions (txt)

In [ ]:
#print(s.decisions)

In [ ]:
# Get just the `snowIncept` option
#print(s.decisions['snowIncept'])
#print(s.decisions['windPrfile'])

print(s.decisions['windPrfile'].available_options)

# Change the value of a decision example
#s.decisions['snowIncept'] = 'stickySnow'
#print(s.decisions['snowIncept'])